In [ ]:
# %% Cargar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, remez, lfilter, freqz, butter, cheby1, group_delay
from IPython.display import display, Audio
import ipywidgets as widgets
from ipywidgets import Layout, HBox, VBox, Output

# %% Contenedor para resultados SNR
snr_output = widgets.HTML("", layout=Layout(padding="10px", border="1px solid #ccc", width="100%", background="#eef8ff"))

# %% Tooltip tipo popup para ayudas
tooltip_popup = Output(layout=Layout(border="1px solid #ccc", padding="8px", width="300px", background="#fffff5", display="none", position="absolute", z_index="100"))

# %% Diccionario de ayudas interactivas
tooltip_texts = {
    "fs": "<b>ℹ Frecuencia de muestreo (Hz):</b><br>Rango permitido: 8000-48000.<br><i>Controla la resolución temporal del sistema.</i>",
    "structure": "<b>ℹ Estructura del filtro:</b><br><b>FIR:</b> Fase lineal, buena estabilidad.<br><b>IIR:</b> Más eficiente, posible distorsión de fase.",
    "method": "<b>ℹ Método de diseño:</b><br><b>FIR:</b> window o remez.<br><b>IIR:</b> butter o cheby1.",
    "type": "<b>ℹ Tipo de filtro:</b><br>Pasa bajas, pasa altas, banda y elimina banda.",
    "f1": "<b>⚠ Frecuencia de corte inferior (f1):</b><br>Mínimo: 20 Hz.<br>Debe ser menor que fs/2.",
    "f2": "<b>⚠ Frecuencia de corte superior (f2):</b><br>Solo para filtros de banda.<br>Mayor que f1 y menor que fs/2.",
    "order_fir": "<b>⚠ Orden FIR (número de taps):</b><br>Debe ser impar. Recomendado > 30 para 'remez'.",
    "order_iir": "<b>⚠ Orden IIR:</b><br>Recomendado entre 2 y 20 para estabilidad y rendimiento.",
    "window": "<b>ℹ Ventana FIR:</b><br>Funciones de ponderación como Hamming, Hann o Blackman.",
    "ripple": "<b>ℹ Ripple (ondulación):</b><br>Solo para Chebyshev I. Define tolerancia en la banda de paso."
}

# %% Función para mostrar ayudas contextualizadas
def make_control(widget, key):
    btn = widgets.Button(description="?", layout=Layout(width="30px"))
    def on_click(b):
        tooltip_popup.clear_output()
        with tooltip_popup:
            display(widgets.HTML(f"<div style='font-family:sans-serif;font-size:14px;'>{tooltip_texts[key]}</div>"))
        tooltip_popup.layout.display = "block"
    btn.on_click(on_click)
    widget.layout = Layout(width="250px")
    return HBox([widget, btn], layout=Layout(padding="2px"))

# %% Definición de widgets de control
structure_dd = widgets.Dropdown(options=["FIR", "IIR"], value="FIR", description="Estructura:")
method_dd = widgets.Dropdown(options=["window", "remez"], value="window", description="Método:")
type_dd = widgets.Dropdown(options=["lowpass", "highpass", "bandpass", "bandstop"], value="lowpass", description="Tipo:")
fs_sl = widgets.IntSlider(min=8000, max=48000, step=1000, value=44100, description="fs:")
f1_sl = widgets.IntSlider(min=20, max=10000, step=100, value=1000, description="f1:")
f2_sl = widgets.IntSlider(min=20, max=10000, step=100, value=5000, description="f2:")
order_sl = widgets.IntSlider(min=1, max=101, step=1, value=31, description="Orden:")
ripple_sl = widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description="Rp:")
window_dd = widgets.Dropdown(options=["hamming", "hann", "blackman"], value="hamming", description="Ventana:")

# %% Panel lateral de controles
controls_panel = VBox([
    make_control(structure_dd, "structure"),
    make_control(method_dd, "method"),
    make_control(type_dd, "type"),
    make_control(fs_sl, "fs"),
    make_control(f1_sl, "f1"),
    make_control(f2_sl, "f2"),
    make_control(order_sl, "order_fir"),
    make_control(ripple_sl, "ripple"),
    make_control(window_dd, "window"),
    snr_output,
    tooltip_popup
], layout=Layout(width="30%"))

# %% Función principal de simulación de filtros
def simulate_filter(structure, method, filt_type, fs, f1, f2, order, ripple=1.0, window="hamming"):
    nyq = fs / 2
    snr_output.value = ""
    tooltip_popup.clear_output()

    if f1 >= nyq or (filt_type in ["bandpass", "bandstop"] and (f2 <= f1 or f2 >= nyq)):
        with tooltip_popup:
            display(widgets.HTML("<b style='color:red;'>Error:</b> Frecuencias de corte no válidas."))
        return

    try:
        if structure == "FIR":
            if order % 2 == 0:
                order += 1
            if method == "window":
                b = firwin(order, [f1, f2] if filt_type in ["bandpass", "bandstop"] else f1,
                           pass_zero=(filt_type in ["lowpass", "bandstop"]), fs=fs, window=window)
            else:  # remez
                if order < 31:
                    with tooltip_popup:
                        display(widgets.HTML("<b style='color:red;'>Error:</b> Para 'remez', use un orden >= 31."))
                    return
                trans = max(100, 0.05 * nyq)
                if filt_type == "lowpass":
                    bands = [0, f1, f1 + trans, nyq]
                    desired = [1, 0]
                elif filt_type == "highpass":
                    bands = [0, f1 - trans, f1, nyq]
                    desired = [0, 1]
                elif filt_type == "bandpass":
                    bands = [0, f1 - trans, f1, f2, f2 + trans, nyq]
                    desired = [0, 1, 0]
                else:
                    bands = [0, f1, f1 + trans, f2 - trans, f2, nyq]
                    desired = [1, 0, 1]
                b = remez(order, bands, desired, fs=fs)
            a = 1.0
        else:
            if order < 2:
                with tooltip_popup:
                    display(widgets.HTML("<b style='color:red;'>Error:</b> El orden IIR debe ser al menos 2."))
                return
            wn = [f1 / nyq] if filt_type in ["lowpass", "highpass"] else [f1 / nyq, f2 / nyq]
            if np.any(np.array(wn) <= 0) or np.any(np.array(wn) >= 1):
                with tooltip_popup:
                    display(widgets.HTML("<b style='color:red;'>Error:</b> Las frecuencias deben estar en el rango (0, fs/2)."))
                return
            if method == "butter":
                b, a = butter(order, wn, btype=filt_type)
            elif method == "cheby1":
                b, a = cheby1(order, ripple, wn, btype=filt_type)

    except Exception as e:
        with tooltip_popup:
            display(widgets.HTML(f"<b style='color:red;'>Error:</b> {str(e)}"))
        return

    # Señal y respuesta
    t = np.linspace(0, 1, fs, endpoint=False)
    x = np.random.randn(len(t))
    y = lfilter(b, a, x)

    fig, axs = plt.subplots(3, 2, figsize=(14, 12))

    # Magnitud y fase
    w, h = freqz(b, a, worN=1024, fs=fs)
    axs[0, 0].plot(w, 20 * np.log10(np.maximum(np.abs(h), 1e-5)))
    axs[0, 0].set(title="Magnitud (dB)", xlabel="Frecuencia (Hz)"); axs[0, 0].grid(True)
    axs[0, 1].plot(w, np.unwrap(np.angle(h)) * 180 / np.pi)
    axs[0, 1].set(title="Fase (°)", xlabel="Frecuencia (Hz)"); axs[0, 1].grid(True)

    # Respuesta al impulso
    imp = b if structure == "FIR" else lfilter(b, a, np.r_[1, np.zeros(99)])
    axs[1, 0].stem(imp, basefmt=" ")
    axs[1, 0].set(title="Respuesta al Impulso", xlabel="Muestras"); axs[1, 0].grid(True)

    # Retardo de grupo
    gd_freq, gd = group_delay((b, a), fs=fs)
    axs[1, 1].plot(gd_freq, gd)
    axs[1, 1].set(title="Retardo de Grupo", xlabel="Frecuencia (Hz)", ylabel="Muestras"); axs[1, 1].grid(True)

    # Señal original vs filtrada
    axs[2, 0].plot(t, x, label="Original")
    axs[2, 0].plot(t, y, label="Filtrada")
    axs[2, 0].set(title="Señal Original vs Filtrada", xlabel="Tiempo (s)")
    axs[2, 0].legend(); axs[2, 0].grid(True)

    # Espectro antes y después
    freqs_fft = np.fft.rfftfreq(len(x), 1/fs)
    axs[2, 1].plot(freqs_fft, np.abs(np.fft.rfft(x)), label="Original")
    axs[2, 1].plot(freqs_fft, np.abs(np.fft.rfft(y)), label="Filtrada")
    axs[2, 1].set(title="Espectro Antes vs Después", xlabel="Frecuencia (Hz)")
    axs[2, 1].legend(); axs[2, 1].grid(True)

    plt.tight_layout(); plt.show()

    # SNR
    try:
        error_signal = x - y
        if np.allclose(error_signal, 0):
            raise ValueError
        snr = 10 * np.log10(np.var(y) / np.var(error_signal))
        snr_output.value = f"<b>SNR:</b> {snr:.2f} dB"
    except:
        snr_output.value = "<b style='color:orange;'>Nota:</b> No se puede calcular el SNR."

    # Audio
    display(Audio(x, rate=fs))
    display(Audio(y, rate=fs))

# %% Función para actualizar controles dinámicos
def update_ui(change=None):
    is_fir = (structure_dd.value == "FIR")
    valid_methods = ["window", "remez"] if is_fir else ["butter", "cheby1"]
    method_dd.options = valid_methods
    if method_dd.value not in valid_methods:
        method_dd.value = valid_methods[0]
    order_sl.description = "Num. taps:" if is_fir else "Orden IIR:"
    ripple_sl.layout.display = "flex" if (not is_fir and method_dd.value == "cheby1") else "none"
    window_dd.layout.display = "flex" if (is_fir and method_dd.value == "window") else "none"
    simulate_filter(
        structure_dd.value, method_dd.value, type_dd.value,
        fs_sl.value, f1_sl.value, f2_sl.value,
        order_sl.value, ripple_sl.value, window_dd.value
    )

# %% Enlazar observadores
structure_dd.observe(update_ui, "value")
method_dd.observe(update_ui, "value")
fs_sl.observe(update_ui, "value")
type_dd.observe(update_ui, "value")
f1_sl.observe(update_ui, "value")
f2_sl.observe(update_ui, "value")

# %% Output interactivo y despliegue
iop = widgets.interactive_output(
    simulate_filter,
    {
        "structure": structure_dd,
        "method": method_dd,
        "filt_type": type_dd,
        "fs": fs_sl,
        "f1": f1_sl,
        "f2": f2_sl,
        "order": order_sl,
        "ripple": ripple_sl,
        "window": window_dd
    }
)

display(controls_panel, iop)